In [1]:
import mdtraj as mdt
import MDAnalysis as mda
from MDAnalysis.lib.util import NamedStream
from io import StringIO

from openpharmacophore.structured_based import StructuredBasedPharmacophore

## Load a trajectory in h5 format

In [2]:
traj_file = "../data/trajectories/ERalpha.h5"
traj = mdt.load(traj_file)
print(traj)
print(traj.topology)

<mdtraj.Trajectory with 1000 frames, 27425 atoms, 8028 residues, and unitcells>
<mdtraj.Topology with 4 chains, 8028 residues, 27425 atoms, 19689 bonds>


In [12]:
type(traj)

mdtraj.core.trajectory.Trajectory

In [3]:
traj.save_xyz("traj.xyz")

## Load trajectory with MDAnalysis

In [3]:
universe = mda.Universe("initial.pdb", "./traj.xyz", in_memory=True)
universe

/home/daniel/miniconda3/envs/cheminformatics/lib/python3.7/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


<Universe with 27425 atoms>

In [13]:
type(universe)

MDAnalysis.core.universe.Universe

In [4]:
universe.trajectory.n_frames

1000

### Write a frame to a stream object and derive a pharmacophore

In [18]:
stream = StringIO()
pdb_stream = NamedStream(stream, "output.pdb")

In [7]:
atoms = universe.select_atoms("all")
atoms.write(pdb_stream, frames=universe.trajectory[[0]])

/home/daniel/miniconda3/envs/cheminformatics/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


In [8]:
pharmacophore = StructuredBasedPharmacophore().from_pdb(pdb_stream, 
                                                        radius=1.0, 
                                                        ligand_id=None, 
                                                        hydrophobics="plip",
                                                        load_mol_system=True,
                                                        load_ligand=True)
pharmacophore

StructuredBasedPharmacophore(n_elements: 9)

In [9]:
pharmacophore.show()

NGLWidget()

## Get pharmacophore for each frame in a trajectory

In [10]:
import time
start_time = time.time()

pharmacophores = []
n_frames = universe.trajectory.n_frames
atoms = universe.select_atoms("all")
for i in range(n_frames):
    stream = StringIO()
    pdb_stream = NamedStream(stream, "output.pdb")
    atoms.write(pdb_stream, frames=universe.trajectory[[i]])
    pharmacophore = StructuredBasedPharmacophore().from_pdb(pdb_stream, 
                                                        radius=1.0, 
                                                        ligand_id=None, 
                                                        hydrophobics="plip",
                                                        load_mol_system=False,
                                                        load_ligand=False)
print("--- %s seconds ---" % (time.time() - start_time))

--- 3354.852271795273 seconds ---


In [11]:
print("Number of pharmacophores: {}".format(len(pharmacophores)))
pharmacophores[0:5]

Number of pharmacophores: 0


[]